# Building a Durak Master

## Step by step creating a Python implementation of the classic Russian card game.
## Game events - attacking and defending (and not losing!)

## Part 2 of 4

In these first two posts I detail the first steps I took to create this game - first purely on the command line and with no AI. The third post will cover machine learning that will build out the opponents, using the Information Set Monte Carlo Tree Search, and the third will focus entirely on bringing a GUI to the game.

This post is part tutorial, but mostly it is my first walk-through of constructing a complete piece of software from scratch. I am happy to hear feedback! Especially feedback around making this code more Pythonic. You can find the full files created in this post (durak.py, game.py, and main.py) on github.com.

Created in Jupyter Notebook - all code is commented to show what went into the main.py game file and what goes into durak.py or game.py.

This post builds directly upon [Part 1](jalexspringer.github.io/durakpt1). Please check it out first! All files can be found in the [github repo](github.com/jalexspringer/durak). 

### A note on organization
I have set aside three modules for the code:
- [main.py](github.com/jalexspringer/durak/main.py) - the game itself. contains some logic checking for win conditions as well as instantiates the objects that make up the game (hands, talon, battlefield, discard, and trump card).
- [durak.py](github.com/jalexspringer/durak/durak.py) - the functions to set up the game (create deck, assign trump, first to play, etc.) and the PlayingCard class.
- [game.py](github.com/jalexspringer/durak/game.py) - primary game loop functions. This is where most of the post today is focused. 

### ```main_game_loop()```

In [ ]:
def main_game_loop(player_count, hands, trump, talon, attacker, defender, battlefield, discard):
    # New turn print of game state
    print_seats(player_count, hands, trump, talon, discard)
    first_attack_hand_string = print_hand(hands[attacker], attacker)
    played = ''
    # First attack
    if len(hands[defender]) > 0 and len(hands[attacker]) > 0:
        played, hands[attacker] = play_card(hands[attacker], attacker, first_attack_hand_string)
        battlefield['attack'].append(played)
    elif len(hands[attacker]) == 0:
        attacker = next_player(attacker, player_count)
        defender = next_player(attacker, player_count)
        return attacker, defender
    elif len(hands[defender]) == 0:
        defender = next_player(defender, player_count)
        return attacker, defender

    # Print state after first attack
    print_all(player_count, hands, trump, talon, battlefield, discard)

    # Defense/attack loop
    return internal_game_loop(player_count, hands, trump, talon, attacker, defender, battlefield, discard, played)


Most of the game is spent inside of this loop. The first attack is unique in that there are no restrictions on which card the attacker can select (other than that it must be in their hand).

The main loop:
- Prints the current state of the game. Seats, hand counts, etc. The attacker's hand is processed to a string and passed to the ```play_card()``` function. This function asks the player for their choice and confirms that it is in the hand and returns the card (```valid_play```) and the ```hand``` minus the card.

In [ ]:
#game.py 
def play_card(hand, active_player, valid_cards, state='attack'):
    """Takes a list of cards and user input, returns the card, and the hand minus the card.
       Runs until a card that is in the hand is input.
       Defaults to the attacker dialogue, set 3rd param to False for defender."""
    while True:
        played = input("Player {}, what card would you like to {} with? ".format(active_player, state))
        for idx, card in enumerate(hand):
            if card.card_name() == played.upper() and played.upper() in valid_cards:
                valid_play = hand.pop(idx)
                return valid_play, hand
        print("That card is not an option. Please select a card from your hand and in the list of playable cards.")

- ```main_game_loop()``` increments the ```attacker``` and ```defender``` if either or both do not have any cards left in their hand, and then returns the new ```attacker``` and ```defender``` values. ```next_player()``` cycles through the players based on the ```player_count``` value.

In [ ]:
#game.py

def next_player(active_player, player_count):
    """Increments active_player to start the next turn
        Also used to assign defender value."""
    if active_player != player_count:
        active_player += 1
    else:
        active_player = 1
    return active_player

- Assuming that the attacker and defender both have cards and the attacker has chosen their attacking card, we add the card to the ```battlefield```, ```print_all()``` prints the battlefield and the current player (defender's) hand. and then step into the ```internal_game_loop()``` that controls the back and forth between players.


### ```internal_game_loop()```

In [ ]:
#game.py
def internal_game_loop(player_count, hands, trump, talon, attacker, defender, battlefield, discard, played, press=True):
    while True:
        # Show legal cards, offer a chance to defend or take all cards
        defending, valid_cards = to_defend(hands[defender], played, trump, defender)  # defend or not to defend
        if defending:
            played, hands[defender] = play_card(hands[defender], defender, valid_cards, 'defend')
            battlefield['defense'].append(played)
            print_all(player_count, hands, trump, talon, battlefield, discard)
            # Check to see if the defender has just played their last card. If yes - end of round. Move to main to check
            # if the talon is empty and defender is the winner.
            if len(hands[defender]) == 0:
                attacker = next_player(attacker, player_count)
                defender = next_player(attacker, player_count)
                return attacker, defender

            # Show legal cards, offer a chance to attack or pass and end attack
            attacking, valid_cards = to_attack(hands[attacker], battlefield, attacker)  # attack
            if attacking:
                played, hands[attacker] = play_card(hands[attacker], attacker, valid_cards)
                battlefield['attack'].append(played)
                print_all(player_count, hands, trump, talon, battlefield, discard)
            else:  # pass
                # If the game has 3 or 4 players, the attack can be continued by the player after the defender
                if player_count > 2 and press:
                    press_attack(player_count, hands, trump, talon, attacker, defender, battlefield, discard, played)
                print("PLAYER {} - DISCARDING CARDS - END OF ATTACK".format(attacker))
                for k, v in battlefield.items():
                    for card in v:
                        discard.append(card)
                attacker = next_player(attacker, player_count)
                defender = next_player(attacker, player_count)
                return attacker, defender
        else:  # take all cards
            additional_throws = throw_in(battlefield, hands, attacker)
            if additional_throws:
                for card in additional_throws:
                    battlefield['attack'].append(card)
            print("PLAYER {} - TAKING CARDS AND SKIPPING YO TURN!!".format(defender))
            for k, v in battlefield.items():
                for card in v:
                    hands[defender].append(card)
            for i in range(2):
                attacker = next_player(attacker, player_count)
                defender = next_player(attacker, player_count)
            return attacker, defender

There is a lot going on there - let's take it step by step.
#### Can/will I defend? - ```to_defend()```
1. The defender is first in the internal loop. ```to_defend()``` checks that they have a valid response to the attack by creating a string of ```valid_cards```, prints the available options, and asks if the defender would like to continue the defense or 'take' the cards in the battlefield.
2. If there is no valid option, or the defender chooses to 'take', ```False``` is returned.
3. If the defender chooses to continue the defense, ```True``` is returned with the string of valid_cards.
4. ```to_defend()``` loops until a valid response is given. ('d' or 'take')

In [ ]:
# game.py
def to_defend(hand, played, trump, defender):
    """Checks that the defender has a playable card, returns playable card options.
        Ask if the defender intends to continue the defense or take the cards"""
    valid_cards = ""
    for card in hand:
        if card.suit == played.suit and card.value > played.value:
            valid_cards += card.card_name() + " "
        elif played.suit == trump.suit and card.suit == trump.suit and card.value > played.value:
            valid_cards += card.card_name() + " "
        elif played.suit != trump.suit and card.suit == trump.suit:
            valid_cards += card.card_name() + " "
    if len(valid_cards) > 0:
        while True:
            print_hand(hand, defender)
            print('Valid card options: {}'.format(valid_cards))
            defending = input("Type 'd' or 'take' to continue: ")
            if defending == 'd':
                return True, valid_cards
            elif defending == 'take':
                return False, valid_cards
            else:
                print('{} is not an option.'.format(defending))
    else:
        print('No valid options - taking cards.')
        return False, valid_cards

#### What card will I defend with?
1. The response from ```to_defend()``` is checked and if ```True```, then ```play_card()``` is called once more - this time for the defender's hand.
2. The chosen card is added to the ```battlefield``` on the defender's side.
3. ```print_all()``` once again prints the game state.
4. Quickly check that the defender has not just played their last card. If yes, end the round and return the next attacker and defender.

If the response from ```to_defend()``` is ```False```, the attacking player may have the chance to throw in additional cards - ```throw_in``` checks for this and gives the attacker the opportunity to select which of these cards they would like to add.



In [ ]:
#game.py
def throw_in(battlefield, hands, attacker):
    """Prompts to see if the attacker would like to add the rest of the cards with a matching value.
        returns the list of cards."""
    valid_cards = ""
    values = []
    cards = []
    for k, v in battlefield.items():
        for card in v:
            if card.value not in values:
                values.append(card.value)
    for idx, card in enumerate(hands[attacker]):
        if card.value in values:
            valid_cards += card.card_name() + " "
    while len(valid_cards) > 0:
        print('Attacking player - you may throw in the rest of the cards in your hand with matching values.')
        print('Valid cards: {}'.format(valid_cards))
        response = input("THROW IN DOUBLES? (y/n)")
        if response == "y":
            played, hands[attacker] = play_card(hands[attacker], attacker, valid_cards, 'throw in')
            cards.append(played)
            valid_cards = valid_cards.replace(played.card_name(), "").strip()
        else:
            break
    return cards

#### Back to the attack!
```to_attack()``` functions much like ```to_defend```, incorporating the rules that govern valid attacking cards. It also checks that there have been 6 or less turns in the round and ends the round otherwise.

In [ ]:
#game.py
def to_attack(hand, battlefield, attacker):
    """Checks that the attacker has a playable card, returns playable card options.
    Ask if the attacker intends to continue the attack or pass"""
    valid_cards = ""
    values = []
    for k, v in battlefield.items():
        for card in v:
            if card.value not in values:
                values.append(card.value)
    for card in hand:
        if card.value in values:
            valid_cards += card.card_name() + " "
    if len(valid_cards) > 0:
        while True:
            print_hand(hand, attacker)
            print('Valid card options: {}'.format(valid_cards))
            attacking = input('Type "a" or "pass" to continue: ')
            if attacking == 'a':
                return True, valid_cards
            elif attacking == 'pass':
                return False, valid_cards
    elif len(battlefield['attack']) >= 6:
        print('Max rounds (6) completed - discarding cards and ending the turn.')
    else:
        print('No valid options - discarding cards and ending the turn.')
        return False, valid_cards

If the attacking player chooses to press the attack:
1. ```play_card()``` is called for the attacker.
2. The chosen card is added to the attacking side of the battlefield.
3. ```print_all()``` displays the current game state.

#### For games with 3 or 4 players
In a multi-player game, if the attacking player declines to continue the attack, the next player after the defender can choose to take over. ```press_attack()``` asks if the player would like to do so and re-enters the ```internal_game_loop()``` to attack if they respond yes.

If the attacking player declines to press the attack:
1. The battlefield cards are added to the discard pile.
2. ```next_player()``` increments and returns the ```attacker``` and ```defender```.


In [ ]:
#game.py
def press_attack(player_count, hands, trump, talon, attacker, defender, battlefield, discard, played):
    """After to_attack() returns False, next attacker has the option to continue the attack
        """
    thrower = next_player(defender, player_count)
    keep_attacking = input('Player {} has passed on the attack.\n'
                           ' Player {} would you like to continue? (y/n) '.format(attacker, thrower))
    if keep_attacking == 'y':
        internal_game_loop(player_count, hands, trump, talon, thrower, defender, battlefield, discard, played)
    else:
        print("PLAYER {} - DISCARDING CARDS - END OF ATTACK".format(thrower))

```internal_game_loop()``` continues until a player declines to attack or defend, or there is no valid card available to a player.

Now that we have the functions defined and the structure of the game built, let's put it all together.

### Running the game
After the initial game setup, the program enters the first turn. A counter is created to keep track of the number of players that have zero cards - the game ends when the counter is equal to ```player_count - 1```. The loser is the player with cards still in her hand.

The structure of each loop is as follows:
1. ```dealer()``` attempts to refill all hands that have less than 6 cards, starting with the attacker. The game cannot end while there are still cards in the talon.
2. If the attacking player has cards in their hand, enter the attack/defense ```main_game_loop()```. If not, skip to the next player.
3. ```main_game_loop()``` returns the next attacker and defender.
4. After the round, reset the battlefield and check for a winner using ```check_win_condition()```.

The code is below. The first block starts the game - I have created a test game with a pre-defined deck and only two suits.

In [1]:
# main.py

from durak import *
from game import *

# GLOBAL CONSTANTS
HAND_SIZE = 6
#  SUITS = ['D', 'C', 'H', 'S']
SUITS = ['H', 'S'] # Limited suits for testing. card_gen() has been modified to not shuffle for consistency

# Starting a game of Durak
print("\nWelcome to Durak!")
player_count = number_of_players()
print("\nThank you - have a good game!")

# Create deck, shuffle into talon, create hands, instantiate battlefield
talon = card_gen(SUITS)
trump = assign_trump(talon)
hands = create_hands(player_count)
battlefield = {'attack': [], 'defense': []}
discard = []

# First deal, find out who goes first, assign initial attacker and defender values
dealer(hands, talon, HAND_SIZE)
attacker, low_card = first_to_play(hands, trump)
defender = next_player(attacker, player_count)
print("Player {} has the lowest trump - {} - and will play first.".format(attacker, low_card))
print("---------------------------------\n")


Welcome to Durak!
How many players? (2, 3, or 4) 2

Thank you - have a good game!
Dealing...
---------------------------------

Player 2 has the lowest trump - 12H - and will play first.
---------------------------------



In [ ]:
#game.py
def check_win_condition(hands):
    cards_in_hand = []
    counter = 0
    for k, v in hands.items():
        cards_in_hand.append(len(v))
    cards_in_hand.sort()
    for i in cards_in_hand:
        if i == 0:
            counter += 1
    return counter

#main.py
counter = 0
while counter < player_count - 1:
    # MAIN GAME LOOP
    dealer(hands, talon, HAND_SIZE)
    if len(hands[attacker]) > 0:
        attacker, defender = main_game_loop(player_count, hands, trump, talon, attacker, defender, battlefield, discard)
        battlefield = {'attack': [], 'defense': []}
        counter = check_win_condition(hands)
    else:
        attacker = next_player(attacker, player_count)
        defender = next_player(attacker, player_count)

loser = 0
for k, v in hands.items():
    if len(v) > 0:
        loser = k
print("Game over! Player {} is the durak and worthy of much derision.".format(loser))

Dealing...
---------------------------------


     Player 2 (6 cards)    
         |        
         |        
         |        
         |        
     Player 1 (6 cards)    

Remaining cards: 6
Trump card: 6H
Total discard: 0
---------------------------------

Player 2 current hand: 12H  13H  14H  6S  7S  8S  
Player 2, what card would you like to attack with? 12h

     Player 2 (5 cards)    
         |        
         |        
         |        
         |        
     Player 1 (6 cards)    

Remaining cards: 6
Trump card: 6H
Total discard: 0
---------------------------------

Attacks: 12H | 
Defense: 

No valid options - taking cards.
PLAYER 1 - TAKING CARDS AND SKIPPING YO TURN!!
Dealing...
---------------------------------


     Player 2 (6 cards)    
         |        
         |        
         |        
         |        
     Player 1 (7 cards)    

Remaining cards: 5
Trump card: 6H
Total discard: 0
---------------------------------

Player 2 current hand: 11H  13H  14

### Game Logic Completed
I won't play through an entire game here - you can see from above that the basic attacking and defending functions work as expected, and if you would like to play through the entire game, please check out the completed files from this post on [github](github.com/jalexspringer/durak/secondpost).

Of course, you'll have to play against yourself or a friend on the command line! Besides the obvious challenge of not peeking at your opponent's cards, the UI isn't exactly appealing. The next post will implement the GUI (real cards!), and the one after that will focus on the AI.